In [203]:
import pandas as pd
import numpy as np
import re

In [204]:
df = pd.read_excel('C:/Users/mirhu/Downloads/data_cleaning.xlsx',sheet_name = 'Usage')
df.head()

,3 PM Mon 24th-Mar-2014___0.384 kwh
0,5AM 15-Aug-2014___1.201 kwh
1,__8PM Thu 20-Mar-2014____1.523 kwh
2,6PM 23rd-Apr-2014___0.424 kwh
3,_1AM Friday 19th-Dec-2014___0.209 kwh
4,_5AM Tue 19th-Aug-2014___1.228 kwh


In [205]:
row = pd.DataFrame({'row':[' 3 PM  Mon 24th-Mar-2014___0.384 kwh  ']})

In [206]:
df.rename({' 3 PM  Mon 24th-Mar-2014___0.384 kwh  ':'row'},axis =1 , inplace = True)

In [207]:
df = pd.concat([row, df] , axis = 0)

In [208]:
df = df.reset_index().drop('index' , axis = 1)

In [209]:
df

,row
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Aug-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Apr-2014___0.424 kwh
4,_1AM Friday 19th-Dec-2014___0.209 kwh
...,...
8755,_1AM Fri 07th-Nov-2014_0.084 kwh
8756,_6AM 20-May-2014__1.027 kwh
8757,__2 AM Tuesday 8th-Apr-2014___0.052 kwh
8758,9 PM 27th-Jan-2014_1.428 kwh


In [210]:
#Replacing all underscores with spaces
df['row'] = df['row'].astype(str).str.replace('_', ' ').str.strip()
df

,row
0,3 PM Mon 24th-Mar-2014 0.384 kwh
1,5AM 15-Aug-2014 1.201 kwh
2,8PM Thu 20-Mar-2014 1.523 kwh
3,6PM 23rd-Apr-2014 0.424 kwh
4,1AM Friday 19th-Dec-2014 0.209 kwh
...,...
8755,1AM Fri 07th-Nov-2014 0.084 kwh
8756,6AM 20-May-2014 1.027 kwh
8757,2 AM Tuesday 8th-Apr-2014 0.052 kwh
8758,9 PM 27th-Jan-2014 1.428 kwh


In [211]:
#Finding and separating the necessary parts of data that will be used
Time, Day, Date, Usage_kwh = [], [], [], []

for i in df['row']:
    time = re.findall("^\d+\ *[A-Z]+", i)
    Time.append(time)

    day = re.findall("[A-Z]+[a-z]+\s", i)
    Day.append(day)
    
    date = re.findall("\d+[a-z]*\-[A-Z]+[a-z]{2}\-[0-9]{4}", i)
    Date.append(date)
    
    usage = re.findall("[0-9]\.\d+", i)
    Usage_kwh.append(usage)

In [212]:
#Gather all them within one dataframe
data_list = zip(Time, Day, Date, Usage_kwh)
column_names = ('time','day', 'date', 'usage')
df_result = pd.DataFrame(data_list, columns=column_names)

df_result

,time,day,date,usage
0,[3 PM],[Mon ],[24th-Mar-2014],[0.384]
1,[5AM],[],[15-Aug-2014],[1.201]
2,[8PM],[Thu ],[20-Mar-2014],[1.523]
3,[6PM],[],[23rd-Apr-2014],[0.424]
4,[1AM],[Friday ],[19th-Dec-2014],[0.209]
...,...,...,...,...
8755,[1AM],[Fri ],[07th-Nov-2014],[0.084]
8756,[6AM],[],[20-May-2014],[1.027]
8757,[2 AM],[Tuesday ],[8th-Apr-2014],[0.052]
8758,[9 PM],[],[27th-Jan-2014],[1.428]


In [213]:
#As the results returned as separate  lists, converting them to pandas series
time = df_result['time'].apply(pd.Series, index = ['time'])
day = df_result['day'].apply(pd.Series)
date =  df_result['date'].apply(pd.Series,  index = ['date'])
usage =  df_result['usage'].apply(pd.Series,  index = ['usage_kWh'])

C:\Users\mirhu\AppData\Local\Temp\ipykernel_71636\1118677032.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  day = df_result['day'].apply(pd.Series)


In [214]:
#Then concatting them again to gather within one dataframe
df = pd.concat([time, day, date, usage], axis=1)
df.rename(columns={ df.columns[1]: "day" }, inplace = True)
df

,time,day,date,usage_kWh
0,3 PM,Mon,24th-Mar-2014,0.384
1,5AM,NaN,15-Aug-2014,1.201
2,8PM,Thu,20-Mar-2014,1.523
3,6PM,NaN,23rd-Apr-2014,0.424
4,1AM,Friday,19th-Dec-2014,0.209
...,...,...,...,...
8755,1AM,Fri,07th-Nov-2014,0.084
8756,6AM,NaN,20-May-2014,1.027
8757,2 AM,Tuesday,8th-Apr-2014,0.052
8758,9 PM,NaN,27th-Jan-2014,1.428


In [215]:
#There are spaces between the numbers and AM/PM in some rows, removing those spaces
df['time'] = df['time'].astype(str).str.replace(' ', '')
df

,time,day,date,usage_kWh
0,3PM,Mon,24th-Mar-2014,0.384
1,5AM,NaN,15-Aug-2014,1.201
2,8PM,Thu,20-Mar-2014,1.523
3,6PM,NaN,23rd-Apr-2014,0.424
4,1AM,Friday,19th-Dec-2014,0.209
...,...,...,...,...
8755,1AM,Fri,07th-Nov-2014,0.084
8756,6AM,NaN,20-May-2014,1.027
8757,2AM,Tuesday,8th-Apr-2014,0.052
8758,9PM,NaN,27th-Jan-2014,1.428


In [216]:
#Splitting 'date' column into day, month and year
df[['Day','Months', 'Year']] = df.date.str.split("-",expand=True) 
df

,time,day,date,usage_kWh,Day,Months,Year
0,3PM,Mon,24th-Mar-2014,0.384,24th,Mar,2014
1,5AM,NaN,15-Aug-2014,1.201,15,Aug,2014
2,8PM,Thu,20-Mar-2014,1.523,20,Mar,2014
3,6PM,NaN,23rd-Apr-2014,0.424,23rd,Apr,2014
4,1AM,Friday,19th-Dec-2014,0.209,19th,Dec,2014
...,...,...,...,...,...,...,...
8755,1AM,Fri,07th-Nov-2014,0.084,07th,Nov,2014
8756,6AM,NaN,20-May-2014,1.027,20,May,2014
8757,2AM,Tuesday,8th-Apr-2014,0.052,8th,Apr,2014
8758,9PM,NaN,27th-Jan-2014,1.428,27th,Jan,2014


In [217]:
#Splitting 'time' column
df[['Time', 'AM/PM']] = df.time.str.split(r"[A-Z]{2}",expand=True) 
df

,time,day,date,usage_kWh,Day,Months,Year,Time,AM/PM
0,3PM,Mon,24th-Mar-2014,0.384,24th,Mar,2014,3,
1,5AM,NaN,15-Aug-2014,1.201,15,Aug,2014,5,
2,8PM,Thu,20-Mar-2014,1.523,20,Mar,2014,8,
3,6PM,NaN,23rd-Apr-2014,0.424,23rd,Apr,2014,6,
4,1AM,Friday,19th-Dec-2014,0.209,19th,Dec,2014,1,
...,...,...,...,...,...,...,...,...,...
8755,1AM,Fri,07th-Nov-2014,0.084,07th,Nov,2014,1,
8756,6AM,NaN,20-May-2014,1.027,20,May,2014,6,
8757,2AM,Tuesday,8th-Apr-2014,0.052,8th,Apr,2014,2,
8758,9PM,NaN,27th-Jan-2014,1.428,27th,Jan,2014,9,


In [218]:
#Bringing AM and PM to the AM/PM column
df[[  'Time1', 'AM/PM']] = df.time.str.split(r"\d+",expand=True) 
df

,time,day,date,usage_kWh,Day,Months,Year,Time,AM/PM,Time1
0,3PM,Mon,24th-Mar-2014,0.384,24th,Mar,2014,3,PM,
1,5AM,NaN,15-Aug-2014,1.201,15,Aug,2014,5,AM,
2,8PM,Thu,20-Mar-2014,1.523,20,Mar,2014,8,PM,
3,6PM,NaN,23rd-Apr-2014,0.424,23rd,Apr,2014,6,PM,
4,1AM,Friday,19th-Dec-2014,0.209,19th,Dec,2014,1,AM,
...,...,...,...,...,...,...,...,...,...,...
8755,1AM,Fri,07th-Nov-2014,0.084,07th,Nov,2014,1,AM,
8756,6AM,NaN,20-May-2014,1.027,20,May,2014,6,AM,
8757,2AM,Tuesday,8th-Apr-2014,0.052,8th,Apr,2014,2,AM,
8758,9PM,NaN,27th-Jan-2014,1.428,27th,Jan,2014,9,PM,


In [219]:
#Removing unnecessary columns
df.drop(columns =["time", "date", "Time1", "day"], inplace = True) 
df

,usage_kWh,Day,Months,Year,Time,AM/PM
0,0.384,24th,Mar,2014,3,PM
1,1.201,15,Aug,2014,5,AM
2,1.523,20,Mar,2014,8,PM
3,0.424,23rd,Apr,2014,6,PM
4,0.209,19th,Dec,2014,1,AM
...,...,...,...,...,...,...
8755,0.084,07th,Nov,2014,1,AM
8756,1.027,20,May,2014,6,AM
8757,0.052,8th,Apr,2014,2,AM
8758,1.428,27th,Jan,2014,9,PM


In [220]:
#Renaming columns
df.rename(columns={'Day': 'month_day', 'Months': 'months', 'Year': 'year', 'Time': 'time', 'AM/PM': 'am_pm'}, inplace=True)
df

,usage_kWh,month_day,months,year,time,am_pm
0,0.384,24th,Mar,2014,3,PM
1,1.201,15,Aug,2014,5,AM
2,1.523,20,Mar,2014,8,PM
3,0.424,23rd,Apr,2014,6,PM
4,0.209,19th,Dec,2014,1,AM
...,...,...,...,...,...,...
8755,0.084,07th,Nov,2014,1,AM
8756,1.027,20,May,2014,6,AM
8757,0.052,8th,Apr,2014,2,AM
8758,1.428,27th,Jan,2014,9,PM


In [221]:
#As some of the variables in column 'month_day' contain letters, removing  them
df['month_day'] = df['month_day'].astype(str).str.replace(r'[a-z]{2}', '')
df

C:\Users\mirhu\AppData\Local\Temp\ipykernel_71636\4116226420.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['month_day'] = df['month_day'].astype(str).str.replace(r'[a-z]{2}', '')


,usage_kWh,month_day,months,year,time,am_pm
0,0.384,24,Mar,2014,3,PM
1,1.201,15,Aug,2014,5,AM
2,1.523,20,Mar,2014,8,PM
3,0.424,23,Apr,2014,6,PM
4,0.209,19,Dec,2014,1,AM
...,...,...,...,...,...,...
8755,0.084,07,Nov,2014,1,AM
8756,1.027,20,May,2014,6,AM
8757,0.052,8,Apr,2014,2,AM
8758,1.428,27,Jan,2014,9,PM


In [222]:
df['month_day'] = df['month_day'].apply(lambda x: x.zfill(2))
df

,usage_kWh,month_day,months,year,time,am_pm
0,0.384,24,Mar,2014,3,PM
1,1.201,15,Aug,2014,5,AM
2,1.523,20,Mar,2014,8,PM
3,0.424,23,Apr,2014,6,PM
4,0.209,19,Dec,2014,1,AM
...,...,...,...,...,...,...
8755,0.084,07,Nov,2014,1,AM
8756,1.027,20,May,2014,6,AM
8757,0.052,08,Apr,2014,2,AM
8758,1.428,27,Jan,2014,9,PM


In [223]:
#Arranging the order of columns
df = df[['time', 'am_pm', 'month_day', 'months', 'year', 'usage_kWh']]
df

,time,am_pm,month_day,months,year,usage_kWh
0,3,PM,24,Mar,2014,0.384
1,5,AM,15,Aug,2014,1.201
2,8,PM,20,Mar,2014,1.523
3,6,PM,23,Apr,2014,0.424
4,1,AM,19,Dec,2014,0.209
...,...,...,...,...,...,...
8755,1,AM,07,Nov,2014,0.084
8756,6,AM,20,May,2014,1.027
8757,2,AM,08,Apr,2014,0.052
8758,9,PM,27,Jan,2014,1.428


In [224]:
#Making all numbers as double digits in 'time' column
df['time'] = df['time'].apply(lambda x: x.zfill(2))
df

,time,am_pm,month_day,months,year,usage_kWh
0,03,PM,24,Mar,2014,0.384
1,05,AM,15,Aug,2014,1.201
2,08,PM,20,Mar,2014,1.523
3,06,PM,23,Apr,2014,0.424
4,01,AM,19,Dec,2014,0.209
...,...,...,...,...,...,...
8755,01,AM,07,Nov,2014,0.084
8756,06,AM,20,May,2014,1.027
8757,02,AM,08,Apr,2014,0.052
8758,09,PM,27,Jan,2014,1.428


In [225]:
#Creating date column
df['date']=df['month_day'].astype(str)+'-'+df['months']+'-'+df['year']
df

,time,am_pm,month_day,months,year,usage_kWh,date
0,03,PM,24,Mar,2014,0.384,24-Mar-2014
1,05,AM,15,Aug,2014,1.201,15-Aug-2014
2,08,PM,20,Mar,2014,1.523,20-Mar-2014
3,06,PM,23,Apr,2014,0.424,23-Apr-2014
4,01,AM,19,Dec,2014,0.209,19-Dec-2014
...,...,...,...,...,...,...,...
8755,01,AM,07,Nov,2014,0.084,07-Nov-2014
8756,06,AM,20,May,2014,1.027,20-May-2014
8757,02,AM,08,Apr,2014,0.052,08-Apr-2014
8758,09,PM,27,Jan,2014,1.428,27-Jan-2014


In [226]:
#Converting 'date' column to date format then getting days of week accroding to the dates
df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.day_name()
df

,time,am_pm,month_day,months,year,usage_kWh,date,day_of_week
0,03,PM,24,Mar,2014,0.384,2014-03-24,Monday
1,05,AM,15,Aug,2014,1.201,2014-08-15,Friday
2,08,PM,20,Mar,2014,1.523,2014-03-20,Thursday
3,06,PM,23,Apr,2014,0.424,2014-04-23,Wednesday
4,01,AM,19,Dec,2014,0.209,2014-12-19,Friday
...,...,...,...,...,...,...,...,...
8755,01,AM,07,Nov,2014,0.084,2014-11-07,Friday
8756,06,AM,20,May,2014,1.027,2014-05-20,Tuesday
8757,02,AM,08,Apr,2014,0.052,2014-04-08,Tuesday
8758,09,PM,27,Jan,2014,1.428,2014-01-27,Monday


In [227]:
#Sort values according to the date and time
df.sort_values(by = ['date', 'am_pm', 'time'], inplace = True)

In [228]:
df

,time,am_pm,month_day,months,year,usage_kWh,date,day_of_week
6316,01,AM,01,Jan,2014,0.34,2014-01-01,Wednesday
5068,02,AM,01,Jan,2014,0.332,2014-01-01,Wednesday
4106,03,AM,01,Jan,2014,0.396,2014-01-01,Wednesday
3239,04,AM,01,Jan,2014,0.388,2014-01-01,Wednesday
1815,05,AM,01,Jan,2014,2.354,2014-01-01,Wednesday
...,...,...,...,...,...,...,...,...
7959,08,PM,31,Dec,2014,1.901,2014-12-31,Wednesday
7703,09,PM,31,Dec,2014,0.766,2014-12-31,Wednesday
1860,10,PM,31,Dec,2014,0.633,2014-12-31,Wednesday
3865,11,PM,31,Dec,2014,0.60,2014-12-31,Wednesday


In [230]:
df['usage_kWh'] = df['usage_kWh'].astype('float')

# Question 1

Answer : B

In [231]:
round(df["usage_kWh"].mean(), 3)

0.782

# Question 2

Answer : D

In [232]:
df.groupby('months')['usage_kWh'].mean().sort_values()

months
Apr    0.210355
Nov    0.322806
Oct    0.421665
Sep    0.485011
May    0.560987
Mar    0.566562
Dec    0.763689
Feb    0.833735
Aug    0.852304
Jun    1.255096
Jan    1.401425
Jul    1.687300
Name: usage_kWh, dtype: float64

# Question 3

Answer: A

In [233]:
df.groupby('day_of_week')['usage_kWh'].mean().sort_values()

day_of_week
Tuesday      0.724843
Monday       0.725894
Wednesday    0.733699
Thursday     0.746375
Friday       0.753666
Saturday     0.890300
Sunday       0.898601
Name: usage_kWh, dtype: float64

# Question 4

Answer: A

In [234]:
df['usage_kWh'].rolling(4).sum().max()

17.237000000000002

# Question 5 

Answer : B

In [237]:
df1 = pd.read_excel (r'C:/Users/mirhu/Downloads/data_cleaning.xlsx', sheet_name='Contracts', index_col=None, 
                     usecols = "A:C, E:G, I:K",  header = 0, keep_default_na=False, skiprows=10)
df1

,No Flex,Unnamed: 1,Unnamed: 2,Monthly Flex,Unnamed: 5,Unnamed: 6,Hourly Flex,Unnamed: 9,Unnamed: 10
0,,,,,,,,,
1,Flat Rate,0.21,$/kWh,January,0.2,$/kWh,12AM until 1AM,0.1,$/kWh
2,,,,February,0.19,$/kWh,1AM until 2AM,0.1,$/kWh
3,,,,March,0.17,$/kWh,2AM until 3AM,0.1,$/kWh
4,,,,April,0.18,$/kWh,3AM until 4AM,0.1,$/kWh
5,,,,May,0.22,$/kWh,4AM until 5AM,0.15,$/kWh
6,,,,June,0.27,$/kWh,5AM until 6AM,0.2,$/kWh
7,,,,July,0.24,$/kWh,6AM until 7AM,0.24,$/kWh
8,,,,August,0.19,$/kWh,7AM until 8AM,0.24,$/kWh
9,,,,September,0.18,$/kWh,8AM until 9AM,0.26,$/kWh


In [240]:
a = df.groupby('months')['usage_kWh'].sum()
a

months
Apr     151.666
Aug     635.819
Dec     568.948
Feb     559.436
Jan    1042.660
Jul    1255.351
Jun     899.904
Mar     422.655
May     416.813
Nov     232.420
Oct     313.719
Sep     348.723
Name: usage_kWh, dtype: float64

In [241]:
b = list(df1.loc[1:12, 'Unnamed: 5'])

myorder = [3, 7, 11, 1, 0, 6, 5, 2, 4, 10, 9, 8]
b = [b[i] for i in myorder]
print(b)

[0.18, 0.19, 0.19, 0.19, 0.2, 0.24, 0.27, 0.17, 0.22, 0.14, 0.15, 0.18]


In [242]:
#Finding annual cost under 'Monthly Flex' contract
monthly_cost = [x * y for x, y in zip(a, b)]

sum(monthly_cost)

1421.2057699999998

# Question 6

Answer: C

In [247]:
a = df['usage_kWh'].sum()
b = 0.21
noflex_cost = a*b
noflex_cost

1438.10394

In [244]:
list2 = df.groupby(['am_pm','time'])['usage_kWh'].sum()
list2

am_pm  time
AM     01       93.966
       02       92.192
       03       92.315
       04       93.596
       05      354.582
       06      446.288
       07      399.243
       08      268.043
       09      190.439
       10      183.174
       11      213.951
       12      181.638
PM     01      184.716
       02      208.166
       03      220.122
       04      219.374
       05      481.243
       06      619.418
       07      731.126
       08      670.779
       09      223.139
       10      210.731
       11      257.327
       12      212.546
Name: usage_kWh, dtype: float64

In [245]:
list3 = list(df1.loc[1:24, 'Unnamed: 9'])
myorder = [1,2,3,4,5,6,7,8,9,10,11,0, 13,14,15,16,17,18,19,20,21,22,23,12]
list3 = [list3[i] for i in myorder]
print(list3)

[0.1, 0.1, 0.1, 0.15, 0.2, 0.24, 0.24, 0.26, 0.2, 0.2, 0.26, 0.1, 0.2, 0.24, 0.18, 0.15, 0.3, 0.24, 0.24, 0.12, 0.11, 0.1, 0.1, 0.26]


In [246]:
hourly_cost = [x * y for x, y in zip(list2, list3)]

sum(hourly_cost)

1368.9764700000003